In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
correct = [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
data = pd.read_csv("C:/Users/yongc/Desktop/math_zero_shot_CoT2.csv", encoding = "utf-8")

In [ ]:
data["correct"] = correct

In [ ]:
data = data[["problem", "level", "type", "solution", "zero-shot CoT reasoning chain", "zero-shot CoT reasoning answer", "correct"]]

In [ ]:
data[data["correct"] == 1].groupby("type").count()["correct"] / data.groupby("type").count()["correct"]

In [ ]:
type_arr = ["Algebra", "Number Theory", "Probability", "Precalculus", "Geometry"]
acc = [25.42, 15.79, 11.11, 10.53, 10]

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(10,6))
plt.bar(type_arr, np.array(acc)/100, color = "orange")
plt.ylabel("Accuracy")
plt.title("Accuracy Distribution for Different Math Sub-categories")
plt.show()

In [ ]:
type_arr = ["level 1", "level 2", "level 3", "level 4", "level 5"]
acc = [41.70, 21.70, 18.60, 5, 5]
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(10,6))
plt.bar(type_arr, np.array(acc)/100, color = "orange")
plt.ylabel("Accuracy")
plt.title("Accuracy Distribution for Different Levels of Difficulty")
plt.show()

In [ ]:
data[data["type"] == "Number Theory"]["solution"].iloc[15]

In [ ]:
drug_folder_path = "C:/Users/yongc/Desktop/drugChat_promptEng/dataset/drugDataset"
drug_info_dict = {}
for filename in tqdm(os.listdir(drug_folder_path)):
    file_path = os.path.join(drug_folder_path, filename)
    with open(file_path, "r", encoding = 'utf-8') as f:
        curr_data = f.read()
    smile_repr_str = get_smile_repr(curr_data)
    drug_info_dict[smile_repr_str] = parse_sections(curr_data, root = smile_repr_str, layer = 2)
    f.close()

In [ ]:
import os
problems = []
answers = []
folder_path = "C:/Users/yongc/Desktop/mathematics_dataset-v1.0/train-medium"
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, "r", encoding = "utf-8") as f:
        for i in range(4):
            if i % 2 == 0:
                problems.append(f.readline().strip())
            else:
                answers.append(f.readline().strip())

In [ ]:
df = pd.DataFrame()
df["problems"] = problems
df["answers"] = answers

In [ ]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
gpt_answers = []

In [ ]:
from tqdm import tqdm
for i in tqdm(range(108, len(problems))):
    try:
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": problems[i] + " Output the answer directly. Do not contain any words other than the answer."}],
          temperature=0,
          max_tokens=256,
        )
        resp = responses.choices[0].message.content
        gpt_answers.append((i, resp))
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except Exception as e:
        print(e)
        time.sleep(10)
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": problems[i]} + " Output the answer directly. Do not contain any words other than the answer."],
          temperature=0,
          max_tokens=256,
        )
        resp = responses.choices[0].message.content
        gpt_answers.append((i, resp))

In [ ]:
df["zero-shot answer"] = [e[1] for e in gpt_answers]

In [ ]:
i = 11
print(df.iloc[i * 10: i * 10 + 10][["answers", "zero-shot answer"]])

In [ ]:
zero_correct = [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
df["zero_answer_correct"] = zero_correct

In [ ]:
np.mean(zero_correct)

In [ ]:
gpt_answers_CoT = []

In [ ]:
from tqdm import tqdm
for i in tqdm(range(len(problems))):
    curr_problem = problems[i] + "\nLet's think step by step."
    try:
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": curr_problem}],
          temperature=0,
          max_tokens=1024,
        )
        resp = responses.choices[0].message.content
        extract_problem = curr_problem + "\n" + resp + "\n" + "Given the problem and the reasoning above, output the final answer directly. Do not contain any words other than the final answer."
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": extract_problem}],
          temperature=0,
          max_tokens=256,
        )
        resp = responses.choices[0].message.content
        gpt_answers_CoT.append((i, resp))
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except Exception as e:
        print(e)
        time.sleep(10)
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": curr_problem}],
          temperature=0,
          max_tokens=2048,
        )
        resp = responses.choices[0].message.content
        extract_problem = curr_problem + "\n" + resp + "\n" + "Given the problem and the reasoning above, output the final answer directly. Do not contain any words other than the final answer."
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": extract_problem}],
          temperature=0,
          max_tokens=256,
        )
        resp = responses.choices[0].message.content
        gpt_answers_CoT.append((i, resp))

In [ ]:
df["CoT answer"] = [e[1] for e in gpt_answers_CoT]

In [ ]:
i = 11
print(df.iloc[i * 10: i * 10 + 10][["answers", "CoT answer"]])

In [ ]:
correct_CoT = [1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]

In [ ]:
np.mean(correct_CoT)

In [ ]:
gpt_answer_explained = []
Explanation = []

In [ ]:
from tqdm import tqdm
import time
for i in tqdm(range(len(problems))):
    curr_problem = problems[i] + "\nExplain the meaning of the problem in details and provide your answer."
    try:
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": curr_problem}],
          temperature=0,
          max_tokens=1024,
        )
        resp = responses.choices[0].message.content
        extract_problem = curr_problem + "\n" + resp + "\n" + "Given the problem and the explanation above, output the final answer directly. Do not contain any words other than the final answer."
        Explanation.append((i, resp))
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": extract_problem}],
          temperature=0,
          max_tokens=256,
        )
        resp = responses.choices[0].message.content
        gpt_answer_explained.append((i, resp))
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except Exception as e:
        print(e)
        time.sleep(10)
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": curr_problem}],
          temperature=0,
          max_tokens=1024,
        )
        resp = responses.choices[0].message.content
        extract_problem = curr_problem + "\n" + resp + "\n" + "Given the problem and the explanation above, output the final answer directly. Do not contain any words other than the final answer."
        Explanation.append((i, resp))
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": extract_problem}],
          temperature=0,
          max_tokens=256,
        )
        resp = responses.choices[0].message.content
        gpt_answer_explained.append((i, resp))

In [ ]:
df["explained"] = [e[1] for e in gpt_answer_explained]

In [ ]:
i = 11
print(df.iloc[i * 10: i * 10 + 10][["answers", "explained"]])

In [ ]:
correct_explained = [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
np.mean(correct_explained)

In [ ]:
PAL_answer = []

In [ ]:
from tqdm import tqdm
import time
for i in tqdm(range(len(problems))):
    curr_problem = problems[i] + "\nWrite a runnable python function that returns the answer. Do not include any text other than python code."
    try:
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": curr_problem}],
          temperature=0,
          max_tokens=1024,
        )
        resp = responses.choices[0].message.content
        try:
            output_stream = io.StringIO()
            sys.stdout = output_stream
            exec(resp)
            printed_output = output_stream.getvalue()
            sys.stdout = sys.__stdout__
            value = int(printed_output.strip())
            PAL_answer.append((i, value))
        except:
            PAL_answer.append(gpt_answers_CoT[i])
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except Exception as e:
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": curr_problem}],
          temperature=0,
          max_tokens=1024,
        )
        resp = responses.choices[0].message.content
        try:
            output_stream = io.StringIO()
            sys.stdout = output_stream
            exec(resp)
            printed_output = output_stream.getvalue()
            
            sys.stdout = sys.__stdout__
            value = int(printed_output.strip())
            PAL_answer.append((i, value))
        except:
            PAL_answer.append(gpt_answers_CoT[i])

In [ ]:
df["PAL"] = [e[1] for e in PAL_answer]

In [ ]:
i = 11
print(df.iloc[i * 10: i * 10 + 10][["answers", "PAL"]])

In [ ]:
correct_PAL = [1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]

In [ ]:
np.mean(correct_PAL)

In [ ]:
from tqdm import tqdm
import time
for i in tqdm(range(len(problems))):
    curr_problem = problems[i] + "\nWrite a runnable python function that returns the answer. Do not include any text other than python code."
    try:
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": curr_problem}],
          temperature=0,
          max_tokens=1024,
        )
        resp = responses.choices[0].message.content
        try:
            output_stream = io.StringIO()
            sys.stdout = output_stream
            exec(resp)
            printed_output = output_stream.getvalue()
            sys.stdout = sys.__stdout__
            value = int(printed_output.strip())
            PAL_answer.append((i, value))
        except:
            PAL_answer.append(gpt_answers_CoT[i])
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except Exception as e:
        responses = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages = [{"role": "user", "content": curr_problem}],
          temperature=0,
          max_tokens=1024,
        )
        resp = responses.choices[0].message.content
        try:
            output_stream = io.StringIO()
            sys.stdout = output_stream
            exec(resp)
            printed_output = output_stream.getvalue()
            
            sys.stdout = sys.__stdout__
            value = int(printed_output.strip())
            PAL_answer.append((i, value))
        except:
            PAL_answer.append(gpt_answers_CoT[i])